In [1]:
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import os

In [ ]:
articles_df = pd.read_csv('shared_articles.csv')
print(articles_df.shape)

In [3]:
base_dir = os.getcwd()
target_dir = os.path.join(base_dir, 'data\essay')

print(target_dir)

c:\learn\star\StarLight\MachineLearning\recommendation\simple-art\data\essay


In [18]:
essays = []

# 读取目标文件夹内所有文件的内容
for file in os.listdir(target_dir):
    with open(os.path.join(target_dir, file), 'r', encoding='utf-8') as f:
        content = f.read()
        cleaned_text = content.replace("[wave]", "")
        parts = content.split('\n\n', 1) 
        print(parts)
        # 最多分割成两部分
        if len(parts) > 1:
            title = parts[0].replace("Title: ", "").strip()  # 移除"Title: "部分并去除两端空白字符
            content = parts[1]  # 内容是第二部分
        else:
            title = None  # 如果没有找到分隔符，那么可能没有标题
            content = parts[0]
        essays.append({ 'title': title, 'content': content})

# 将列表转换为DataFrame
df = pd.DataFrame(essays)

['Title: 地球妈妈，我想对你说', '地球妈妈，我想对你说：“我昨天做了一个梦，梦见‘宇宙急诊室’里有一个病情十分严重的患者，只见医术高明的火星大夫一边急忙地朝急诊室的方向跑去，一边披上白大褂，嘴里还不停地喊着：‘请各位大夫立即前往急诊室会诊。’只见急诊室外有一位母亲正在焦急地来回走动——那就是所有星球的母亲太阳。不一会儿急诊室的灯灭了，火星大夫从里面走了出来，脸上带着几分焦虑，太阳母亲的心顿时提到了嗓子眼儿。火星大夫焦急地说：‘令爱情况不是很好，第一，容颜憔悴。第二，血液变得不新鲜了。第三，咳嗽严重。第四，也是最重要的一点，心理状况不是很好，每天生活在悲哀之中。’太阳问：‘怎么会这样子呢？她每年的‘宇宙选美赛’中可都是冠军呀！’火星大夫气愤地说；‘都是她的那些儿女干的。他们为了造纸大量砍伐树木，据统计1000张纸就需要消耗一棵大树。你想想这一年要多少大树呀！而且人类在河流边建造工厂，把废水排入清澈的河流中，把有毒气体排入空气中，使二氧化碳迅速增多，空气质量变差，年均温度正直线上升，不久的将来人类将面临严重的温室效应。温室效应会使冬天逐渐变热，赤道地区的夏天会变得更旱，秋天变得更湿，森林火险气象等级每年将有大部分地区为一级。南北两极大部分冰川融化，沿海地区将面临严重的泥石流、洪涝等自然灾害。全球大部分森林将会被烧毁，人类将面临灭绝灾难。[wave]’太阳问：‘怎么才能治好她的病呢？’火星大夫说：‘这到不难，只要人类不再排放有毒气体，不乱砍乱伐，保护好生态环境，就能治好令爱的病，也能解救人类自己将面临的危险。’惊醒后，我发现这是一个梦。地球妈妈我们人类终于觉醒了，再也不会伤害你了！”[/wave]']
['Title: 想飞的蚂蚁', '[wave]蔚蓝的天空中，漂浮着几朵洁白的云，三五成群的鸟儿在云朵里捉着迷藏，一行大雁排着整齐的队伍向远处飞去。[/wave][br]这美丽的情景被地上的一只小小的蚂蚁全都看在了眼里，[wave]它的眼里流露出羡慕和向往[/wave]，如果我也有一双翅膀，也能够在天空中自由地飞翔那该多好啊！[br]这只小蚂蚁心中满怀宏伟的梦想，开始充满信心地做试验。它首先拿两片小树叶绑在了手上，想作为翅膀，可是它使劲儿扇动，就是飞不起来，它又多拿了两片小树叶绑在手上，还是飞不起来。[br]它伤心地哭了。这时，小蜜蜂飞来说：“小蚂蚁，你应该先学

In [16]:
df.head()

,title,content
0,地球妈妈，我想对你说,地球妈妈，我想对你说：“我昨天做了一个梦，梦见‘宇宙急诊室’里有一个病情十分严重的患者，只见...
1,想飞的蚂蚁,[wave]蔚蓝的天空中，漂浮着几朵洁白的云，三五成群的鸟儿在云朵里捉着迷藏，一行大雁排着整...
2,我最敬佩的一个人,我最敬佩的一个人是一位环卫工人。他高瘦的身材，圆圆的眼睛，黑黑的脸庞，衣衫褴褛，别看它的外表...
3,夜空,[wave]天，渐渐的暗了下来。太阳也收敛它最后的笑容。夜神用它那深蓝的袍子把整个天空都笼罩...
4,秋菊,早晨，当你踏入校园的第一步时，你会感觉到，一阵清香扑鼻而来，紧接着，一片[wave]红花衬绿...


In [21]:
def create_soup(x):
    soup = ' '.join(x['content'])
    return soup

In [22]:
df['soup'] = df.apply(create_soup, axis=1)

In [23]:
df.head()

,title,content,soup
0,地球妈妈，我想对你说,地球妈妈，我想对你说：“我昨天做了一个梦，梦见‘宇宙急诊室’里有一个病情十分严重的患者，只见...,地 球 妈 妈 ， 我 想 对 你 说 ： “ 我 昨 天 做 了 一 个 梦 ， 梦 见 ...
1,想飞的蚂蚁,[wave]蔚蓝的天空中，漂浮着几朵洁白的云，三五成群的鸟儿在云朵里捉着迷藏，一行大雁排着整...,[ w a v e ] 蔚 蓝 的 天 空 中 ， 漂 浮 着 几 朵 洁 白 的 云 ， ...
2,我最敬佩的一个人,我最敬佩的一个人是一位环卫工人。他高瘦的身材，圆圆的眼睛，黑黑的脸庞，衣衫褴褛，别看它的外表...,我 最 敬 佩 的 一 个 人 是 一 位 环 卫 工 人 。 他 高 瘦 的 身 材 ， ...
3,夜空,[wave]天，渐渐的暗了下来。太阳也收敛它最后的笑容。夜神用它那深蓝的袍子把整个天空都笼罩...,[ w a v e ] 天 ， 渐 渐 的 暗 了 下 来 。 太 阳 也 收 敛 它 最 ...
4,秋菊,早晨，当你踏入校园的第一步时，你会感觉到，一阵清香扑鼻而来，紧接着，一片[wave]红花衬绿...,早 晨 ， 当 你 踏 入 校 园 的 第 一 步 时 ， 你 会 感 觉 到 ， 一 阵 ...


In [30]:
stop_word_list = []

with open("baidu_stopwords.txt", encoding='utf-8') as f:
    stop_content = f.read()
stop_word_list = stop_content.strip().split('\n')

In [38]:
import jieba

def chinese_tokenizer(text):
    return jieba.lcut(text)

In [39]:
tfidf = TfidfVectorizer(tokenizer=chinese_tokenizer, stop_words=stop_word_list)

# 对文本数据进行分词和向量化处理
tfidf_matrix = tfidf.fit_transform(df['content'])

c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'a', 'ain', 'aren', 'c', 'couldn', 'd', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'i', 'isn', 'll', 'm', 'mon', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won', 'wouldn', '下', '不', '为什', '什', '今', '使', '先', '却', '只', '唷', '啪', '喔', '天', '好', '後', '最', '漫', '然', '特', '特别', '见', '设', '说', '达', '面', '麽', '－'] not in stop_words.
  warnings.warn(


In [40]:
print(tfidf_matrix.shape)

(300, 10526)


In [41]:
print(tfidf.get_feature_names_out()[5000:5010])

['挺拔' '挺挺' '挺晚' '挺直' '挺立' '挽' '挽留' '挽起' '捂住' '捂着']


In [42]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix, True)
display(cosine_sim.shape)
display(cosine_sim)

(300, 300)

array([[1.        , 0.17001078, 0.15649181, ..., 0.13185884, 0.17985946,
        0.13284085],
       [0.17001078, 1.        , 0.24830044, ..., 0.22081068, 0.25580306,
        0.2139231 ],
       [0.15649181, 0.24830044, 1.        , ..., 0.20262034, 0.23926343,
        0.18897341],
       ...,
       [0.13185884, 0.22081068, 0.20262034, ..., 1.        , 0.23917648,
        0.20049231],
       [0.17985946, 0.25580306, 0.23926343, ..., 0.23917648, 1.        ,
        0.223998  ],
       [0.13284085, 0.2139231 , 0.18897341, ..., 0.20049231, 0.223998  ,
        1.        ]])

In [44]:
metadata = df.reset_index()
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()
display(indices[:10])

title
地球妈妈，我想对你说      0
想飞的蚂蚁           1
我最敬佩的一个人        2
夜空              3
秋菊              4
寻找春天            5
落叶              6
《和好孩子交朋友》读后感    7
感谢              8
小蚂蚁得救了          9
dtype: int64

In [45]:
def get_recommendations(title, indices, cosine_sim, data):
    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the articles based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the article indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar articles
    return data['title'].iloc[movie_indices]

In [58]:
print(get_recommendations('夜空', indices, cosine_sim
                            , metadata))

238    最美丽的梦
292     一年四季
44      我的家乡
117        春
123        雨
119       雷雨
134      秋姑娘
288        春
6         落叶
72       中秋节
Name: title, dtype: object


In [50]:
def get_recommendations_by_idx(idx, cosine_sim, data, top_n=10, sim_threshold=0.1):
    # 保证索引在cosine_sim的范围内
    if idx >= len(cosine_sim):
        return "Index out of bounds."
    
    # 获取所有文章与该索引文章的相似度得分
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 根据相似度得分排序文章，并移除自身
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    # 应用相似度阈值过滤
    sim_scores = [sim_score for sim_score in sim_scores if sim_score[1] > sim_threshold]

    # 获取最相似文章的索引
    article_indices = [i[0] for i in sim_scores]

    # 返回最相似文章的标题
    recommendations = data['title'].iloc[article_indices]

    return recommendations

# 示例调用
# 假设你已经通过某种方式找到了与关键词最相关的文章索引article_idx
# cosine_sim是文章相似度矩阵，data是包含文章标题的DataFrame
# recommendations = get_recommendations_by_idx(article_idx, cosine_sim, data, top_n=10, sim_threshold=0.2)
# print(recommendations)


In [62]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# 假设df是你的DataFrame，包含了'content'和'title'列
# 以下示例创建一个基于内容的TF-IDF矩阵，并基于内容的相似度来进行推荐

# 创建TF-IDF向量化器
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_word_list)

# 向量化文章内容
tfidf_matrix = tfidf_vectorizer.fit_transform(df['content'])

# 计算余弦相似度矩阵
cosine_sim_content = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 假设有一个查询函数，用于找到与查询关键词最相关的文章
def find_article_by_keyword(keyword, tfidf_vectorizer, df):
    # 将查询关键词转换为TF-IDF向量
    keyword_vector = tfidf_vectorizer.transform([keyword])
    
    # 计算查询关键词与所有文章内容的相似度
    sim_scores_keyword = cosine_similarity(keyword_vector, tfidf_matrix)
    
    # 找到最相似的文章索引
    most_similar_article_idx = sim_scores_keyword.argsort()[0][-1]
    
    return most_similar_article_idx

# 假设关键词是"user experience"
keyword = "夜空"
article_idx = find_article_by_keyword(keyword, tfidf_vectorizer, df)
print(article_idx)

# 使用上述`get_recommendations`函数获取推荐
# 注意，需要稍作修改以接受索引而非标题作为输入
recommendations = get_recommendations_by_idx(article_idx, cosine_sim_content, df)

# 打印推荐结果的标题
print(recommendations)


299
244    爱
Name: title, dtype: object


In [49]:
def get_recommendations(title, indices, cosine_sim, data, top_n=10, sim_threshold=0.1):
    # 检查提供的标题是否存在于索引中
    if title not in indices:
        return f"Title '{title}' not found."

    # 获取文章的索引
    idx = indices[title]

    # 获取所有文章与该文章的相似度得分
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 根据相似度得分排序文章，并移除自身
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    # 应用相似度阈值过滤
    sim_scores = [sim_score for sim_score in sim_scores if sim_score[1] > sim_threshold]

    # 获取最相似文章的索引
    article_indices = [i[0] for i in sim_scores]

    # 获取最相似文章的标题
    recommendations = data['title'].iloc[article_indices]

    return recommendations

In [27]:
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer

# 定义中文停用词集合
chinese_stop_words = {'的', '了', '在', '是', '我', '有', '和', '就'}
chinese_stop_words_list = list(chinese_stop_words)

# 使用转换后的列表创建TfidfVectorizer实例
tfidf = TfidfVectorizer(stop_words=chinese_stop_words_list)

# 示例：使用TfidfVectorizer实例
# 这里只是示例，实际上你需要传入真实的中文文本数据进行向量化
texts = ["我爱北京天安门", "天安门上太阳升"]
tfidf_matrix = tfidf.fit_transform(texts)

print(tfidf_matrix)

  (0, 1)	1.0
  (1, 0)	1.0


In [25]:
def jieba_tokenize(text):
    return [word for word in jieba.cut(text) if word not in stop_words]

In [31]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=jieba_tokenize, stop_words=stop_word_list)

# 准备一些中文文本进行测试
texts = [
    "我爱北京天安门",
    "天安门上太阳升",
    "伟大领袖毛主席",
    "指引我们向前进"
]

# 使用fit_transform方法进行分词和向量化
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

print(tfidf_matrix)


c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Building prefix dict from the default dictionary ...


Dumping model to file cache C:\Users\18353\AppData\Local\Temp\jieba.cache
Loading model cost 0.756 seconds.
Prefix dict has been built successfully.


  (0, 3)	0.48693426407352264
  (0, 2)	0.6176143709756019
  (0, 7)	0.6176143709756019
  (1, 4)	0.6176143709756019
  (1, 0)	0.6176143709756019
  (1, 3)	0.48693426407352264
  (2, 6)	0.7071067811865476
  (2, 1)	0.7071067811865476
  (3, 5)	1.0


c:\Users\18353\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'a', 'ain', 'aren', 'c', 'couldn', 'd', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'i', 'isn', 'll', 'm', 'mon', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won', 'wouldn', '下', '不', '为什', '什', '今', '使', '先', '却', '只', '唷', '啪', '喔', '天', '好', '後', '最', '漫', '然', '特', '特别', '见', '设', '说', '达', '面', '麽', '－'] not in stop_words.
  warnings.warn(


In [32]:
tfidf_matrix

<4x8 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>